In [1]:
! createdb financial_event

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import logging

PARALLEL = 4 # assuming a quad-core machine
ATTRIBUTE = "financial_event"
conn_string = 'postgresql://localhost:5432/' + ATTRIBUTE

In [3]:
from fonduer import Meta, init_logging

# Configure logging for Fonduer
init_logging(log_dir="logs")

session = Meta.init(conn_string).Session()

[2020-05-09 21:14:11,389][INFO] fonduer.meta:51 - Setting logging directory to: logs/2020-05-09_21-14-11
[2020-05-09 21:14:11,455][INFO] fonduer.meta:135 - Connecting user:None to localhost:5432/financial_event
[2020-05-09 21:14:13,973][INFO] fonduer.meta:162 - Initializing the storage schema


In [76]:
from fonduer.parser.preprocessors import HTMLDocPreprocessor
from fonduer.parser import Parser

In [93]:
docs_path = "data/"
doc_preprocessor = HTMLDocPreprocessor(docs_path)
corpus_parser = Parser(session, language='zh', structural=True, lingual=True)
%time corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)

[2020-05-10 23:16:18,710][INFO] fonduer.utils.udf:68 - Running UDF...


[2020-05-10 23:16:18,906][WARNING] fonduer.parser.parser:224 - Lingual mode will be turned off, as spacy doesn't provide support for this language (zh)
[2020-05-10 23:16:18,910][WARNING] fonduer.parser.parser:224 - Lingual mode will be turned off, as spacy doesn't provide support for this language (zh)
[2020-05-10 23:16:18,913][WARNING] fonduer.parser.parser:224 - Lingual mode will be turned off, as spacy doesn't provide support for this language (zh)
[2020-05-10 23:16:18,915][WARNING] fonduer.parser.parser:224 - Lingual mode will be turned off, as spacy doesn't provide support for this language (zh)
Building prefix dict from the default dictionary ...
[2020-05-10 23:16:19,028][DEBUG] jieba:113 - Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/fp/tf276mb96jv23y3jfxvvk29c0000gn/T/jieba.cache
[2020-05-10 23:16:19,037][DEBUG] jieba:133 - Loading model from cache /var/folders/fp/tf276mb96jv23y3jfxvvk29c0000gn/T/jieba.cache
Loading model cost 0.755


CPU times: user 296 ms, sys: 64.2 ms, total: 360 ms
Wall time: 1.85 s


In [94]:
from fonduer.parser.models import Document, Sentence

print(f"Documents: {session.query(Document).count()}")
print(f"Sentences: {session.query(Sentence).count()}")

Documents: 0
Sentences: 0


In [21]:
docs = session.query(Document).order_by(Document.name).all()
ld = len(docs)

train_docs = set()
dev_docs = set()
test_docs = set()
splits = (0.7, 0.85)
data = [(doc.name, doc) for doc in docs]
data.sort(key=lambda x: x[0])
for i, (doc_name, doc) in enumerate(data):
    if i < splits[0] * ld:
        train_docs.add(doc)
    elif i < splits[1] * ld:
        dev_docs.add(doc)
    else:
        test_docs.add(doc)
from pprint import pprint

pprint([x.name for x in train_docs])

['SZ001696_2013-07-12s', 'SH600283_2009-07-21_54901568s']


In [27]:
from fonduer.candidates.models import mention_subclass

Presidentname = mention_subclass("Presidentname")
Placeofbirth = mention_subclass("Placeofbirth")

In [95]:
sens = session.query(Sentence).all()

In [96]:
sens

[]

In [42]:
TextDocPreprocessor?

In [43]:
Parser?

In [52]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [ ]:
def convert(case, pdfpath, targetfilepath, pages=100):
    if not pages: pagenums = set();
    else:         pagenums = set(pages);
    manager = PDFResourceManager()
    codec = 'utf-8'
    caching = True
    word_margin = 1
    laparams2 = LAParams(all_texts=True, detect_vertical=False,
                      line_overlap=0.5, char_margin=10, #set char_margin to a large number
                      line_margin=0.1, word_margin=0.5,
                      boxes_flow=None)

    if case == 'text':
        output = io.StringIO()
        converter = TextConverter(manager, output, codec=codec, laparams=LAParams())
    if case == 'HTML':
        output = io.BytesIO()
        converter = HTMLConverter(manager, output, codec=codec, laparams=LAParams())
    if case == 'XML':
        output = io.BytesIO()
        converter = XMLConverter(manager, output, codec=codec, laparams= laparams2)

    interpreter = PDFPageInterpreter(manager, converter)
    infile = open(pdfpath, 'rb')

    for page in PDFPage.get_pages(infile, pagenums,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    convertedPDF = output.getvalue()

    infile.close(); converter.close(); output.close()

    convertedFile = open(targetfilepath, 'wb')
    convertedFile.write(convertedPDF)
    convertedFile.close()

print(convert('XML', 'data/SH600283_2009-07-21_54901568.pdf', 'data/test.xml', pages=None))

In [68]:
def get_mulu_text1(pdf_path, targetfilepath, pages=100):
    if not pages: 
        pagenums = set();
    else:
        pagenums = set(pages);
    codec = 'utf-8'
    caching = True
    output = io.BytesIO()
    
    resource_manager = PDFResourceManager()
#     page_aggregator = PDFPageAggregator(resource_manager, laparams=LAParams())
    converter = HTMLConverter(resource_manager, output, codec=codec, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    
    with open(pdf_path, 'rb') as fin:
        for page in PDFPage.get_pages(fin, pagenums, caching=caching):
            page_interpreter.process_page(page)
        
    convertedPDF = output.getvalue()
    converter.close(); output.close()

    convertedFile = open(targetfilepath, 'wb')
    convertedFile.write(convertedPDF)
    convertedFile.close()
    
get_mulu_text1('./data/SH600283_2009-07-21_54901568.pdf', './data/again.xml', pages=None)

[2020-05-10 16:25:53,397][INFO] pdfminer.pdfdocument:773 - xref found: pos=b'30743'
[2020-05-10 16:25:53,398][INFO] pdfminer.pdfdocument:785 - read_xref_from: start=30743, token=/b'xref'
[2020-05-10 16:25:53,399][INFO] pdfminer.pdfdocument:117 - xref objects: {1: (None, 2677, 0), 2: (None, 30169, 0), 3: (None, 2609, 0), 4: (None, 2467, 0), 5: (None, 15, 0), 6: (None, 2447, 0), 7: (None, 2742, 0), 8: (None, 5110, 0), 9: (None, 3096, 0), 10: (None, 9882, 0), 11: (None, 3791, 0), 12: (None, 2820, 0), 13: (None, 3637, 0), 14: (None, 2951, 0), 15: (None, 5427, 0), 16: (None, 2899, 0), 17: (None, 5635, 0), 18: (None, 10139, 0), 19: (None, 3228, 0), 20: (None, 3874, 0), 21: (None, 3934, 0), 22: (None, 5262, 0), 23: (None, 9532, 0), 24: (None, 28372, 0)}
[2020-05-10 16:25:53,401][INFO] pdfminer.pdfdocument:799 - trailer: {'Size': 25, 'Root': <PDFObjRef:1>, 'Info': <PDFObjRef:2>, 'ID': [b"\xdc\x04\x0e1\x10\xdb\xb2\xb0\xa2'\xbe~\xc1\x93\x06\x96", b"\xdc\x04\x0e1\x10\xdb\xb2\xb0\xa2'\xbe~\xc1\x93

TypeError: a bytes-like object is required, not 'str'

In [85]:
Parser?